## 読み込み

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from collections import defaultdict

In [2]:
ratings_df = pd.read_csv('/home/ubuntu/MovieLens/ratings.dat', sep='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')
movies_df = pd.read_csv('/home/ubuntu/MovieLens/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'], engine='python', encoding='latin1')
movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|'))

## 前処理

In [3]:
unique_genres = set() # 映画ジャンルの集合（重複なし）
for genres in movies_df['genres']:
    unique_genres.update(genres)

genre2id = {genre: idx for idx, genre in enumerate(unique_genres)} # ジャンルがキー，対応するIDが値

adj_list = defaultdict(set) # 映画とそのジャンルの隣接リスト（辞書），キーが存在しないときは空集合を返す
for _, row in movies_df.iterrows(): 
    movie_id = row['movie_id']
    for genre in row['genres']: # 各映画のジャンルリスト
        genre_id = genre2id[genre] # 各ジャンルに対応するIDを取得
        adj_list[movie_id].add(len(movies_df) + genre_id)
        adj_list[len(movies_df) + genre_id].add(movie_id) # 映画とジャンル間のリンクを隣接リストに追加

for key in adj_list: 
    adj_list[key] = list(map(int, adj_list[key])) # 映画と関連するジャンルのリスト．mapで各要素を整数に

## モデル

In [4]:
class KGAT(nn.Module):
    def __init__(self, num_users, num_items, num_genres, embedding_dim): #初期化関数：ユーザー数，アイテム（映画+ジャンル）数，ジャンル数，埋め込みの次元数
        super(KGAT, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim) # ユーザーの埋め込みベクトル
        self.item_embeddings = nn.Embedding(num_items, embedding_dim) # アイテムの埋め込みベクトル
        
        self.user_attention = nn.Linear(embedding_dim, 1) # ユーザーのAttentionを計算する線形層
        self.item_attention = nn.Linear(embedding_dim, 1) # アイテムのAttentionを計算する線形層

    def forward(self, users, items, adj_list): # ユーザー，アイテム，隣接リスト
        user_embeds = self.user_embeddings(users)
        item_embeds = self.item_embeddings(items)
        
        batch_size = len(items)
        
        aggregated_item_embeds = []
        for i in range(batch_size): # 各アイテム
            neighbors = adj_list[items[i].item()] # アイテムの隣接ノード（ジャンル）を取得
            item_neighbors = torch.tensor(list(neighbors), dtype=torch.long).to(user_embeds.device) # テンソル変換
            item_neighbor_embeds = self.item_embeddings(item_neighbors) # アイテムの隣接ノードの埋め込み
            # ユーザーの埋め込みとの間でAttention計算
            attention = torch.softmax(torch.matmul(user_embeds[i].unsqueeze(0), item_neighbor_embeds.t()), dim=-1)
            # Attentionと隣接ノード（ジャンル）の埋め込みベクトルの内積を計算（各隣接ノード（ジャンル）が出力埋め込みにどの程度寄与するか決定）
            aggregated_item_embed = torch.matmul(attention, item_neighbor_embeds)
            aggregated_item_embeds.append(aggregated_item_embed)
        
        aggregated_item_embeds = torch.cat(aggregated_item_embeds) # バッチ内のすべてのアイテムの集約された埋め込みを連結
        # ユーザーの埋め込みと集約されたアイテム埋め込みの要素ごとの積を計算し，それを合計してスコアを出す．スコアはシグモイド関数で正規化
        preds = torch.sigmoid(torch.sum(user_embeds * aggregated_item_embeds, dim=-1))
        return preds

In [5]:
embedding_dim = 64 # 埋め込みの次元数
num_users = ratings_df['user_id'].max() + 1 # ユーザー数
num_items = len(movies_df) + len(adj_list) # アイテム（映画+ジャンル数）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 学習

In [ ]:
# K分割交差検証
kf = KFold(n_splits=5, shuffle=True, random_state=42)

embedding_dim = 64
num_users = ratings_df['user_id'].max() + 1
num_items = len(movies_df) + len(adj_list)
num_epochs = 10
batch_size = 256
lr = 0.01

models = []

for train_index, _ in kf.split(ratings_df):
    train_df = ratings_df.iloc[train_index]

    user_ids = torch.tensor(train_df['user_id'].values, dtype=torch.long)
    item_ids = torch.tensor(train_df['movie_id'].values, dtype=torch.long)
    ratings = torch.tensor(train_df['rating'].values, dtype=torch.float)

    model = KGAT(num_users, num_items, len(genre2id), embedding_dim)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for i in range(0, len(user_ids), batch_size): # ミニバッチごとに
            batch_user_ids = user_ids[i:i+batch_size].to(device)
            batch_item_ids = item_ids[i:i+batch_size].to(device)
            batch_ratings = ratings[i:i+batch_size].to(device)

            optimizer.zero_grad()
            preds = model(batch_user_ids, batch_item_ids, adj_list) * 5
            loss = criterion(preds, batch_ratings)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(user_ids)}") # 各エポックでの平均損失

    models.append(model)

Epoch 1/10, Loss: 0.01853796561757384
Epoch 2/10, Loss: 0.015087618748630722
Epoch 3/10, Loss: 0.010451089807084313


## 推論
5つのモデルの平均スコアで性能評価

In [ ]:
mse_scores = []

for _, test_index in kf.split(ratings_df):
    test_df = ratings_df.iloc[test_index]
    test_user_ids = torch.tensor(test_df['user_id'].values, dtype=torch.long)
    test_item_ids = torch.tensor(test_df['movie_id'].values, dtype=torch.long)
    test_ratings = torch.tensor(test_df['rating'].values, dtype=torch.float)

    mse_per_model = []
    for model in models: # モデルごとに
        model.eval() # 評価モード
        with torch.no_grad(): # 評価では勾配情報無効に
            test_preds = model(test_user_ids.to(device), test_item_ids.to(device), adj_list) * 5
            mse = mean_squared_error(test_preds.cpu().numpy(), test_ratings.cpu().numpy())
            mse_per_model.append(mse)

    mse_scores.append(np.mean(mse_per_model)) # 各モデルの平均MSE

print(f'Average MSE for {kf.get_n_splits()} folds: {np.mean(mse_scores)}')